In [0]:
# import os

# # Install java
# ! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
# ! java -version

# Install pyspark
! pip3 install --ignore-installed pyspark==2.4.4

# Install Spark NLP
! pip3 install --ignore-installed spark-nlp==2.4.5

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 56kB/s 
     |████████████████████████████████| 204kB 47.0MB/s 
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130388 sha256=b7a8507d64fa3a6132e6c7bca775fbaad00fde383b447685d2ccae5490b6705c
  Stored in directory: /root/.cache/pip/wheels/ab/09/4d/0d184230058e654eb1b04467dbc1292f00eaa186544604b471
Successfully built pyspark
     |████████████████████████████████| 112kB 8.5MB/s 


In [0]:
import findspark
findspark.init('/home/cse587/spark-2.4.0-bin-hadoop2.7')

import sparknlp
spark = sparknlp.start()

print("Spark NLP version")
sparknlp.version()
print("Apache Spark version")
spark.version

Spark NLP version
Apache Spark version


'2.4.4'

In [0]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [0]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [0]:
from pyspark.sql.types import IntegerType,ArrayType, StringType
from pyspark.sql.functions import udf
from pyspark.sql.functions import col
from copy import deepcopy

import ast

In [0]:
file = sqlContext.read.csv('train.csv',header=True)

documents = sc.textFile("train.csv").map(lambda line: line.split(","))

In [0]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover,HashingTF, IDF
from pyspark.ml import Pipeline

preProcStages = []

regexTokenizer = RegexTokenizer(gaps=False, pattern='\w+', inputCol='plot', outputCol='token')
stopWordsRemover = StopWordsRemover(inputCol='token', outputCol='clean')
hashingTF = HashingTF(inputCol="clean", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features", minDocFreq=5)

preProcStages += [regexTokenizer, stopWordsRemover,hashingTF,idf]
  
pipeline = Pipeline(stages=preProcStages)

model = pipeline.fit(file)
data_set = model.transform(file)

In [0]:
import pandas as pd

d = pd.read_csv('mapping.csv',index_col=0)
df = d.to_dict()['0']

dic = dict((v,k) for k,v in df.items())
encode = [0] * len(dic.keys())

In [0]:
#for columns
from ast import literal_eval
from copy import deepcopy

def kHotEncode(x):
    global dic
    global encode

    key = deepcopy(encode)
    try:
        labs = list(literal_eval(x))
        res = []
        for lab in labs:
            if lab in dic.keys(): key[dic[lab]] = 1
        return key
    except:
        return [0] * 20

udf_encode = udf(kHotEncode, ArrayType(IntegerType()))

data_set = data_set.withColumn("label", udf_encode(col("genre")))
data_set.head(1)


[Row(movie_id='23890098', movie_name='Taxi Blues', plot="Shlykov, a hard-working taxi driver and Lyosha, a saxophonist, develop a bizarre love-hate relationship, and despite their prejudices, realize they aren't so different after all.", genre="['World cinema', 'Drama']", token=['shlykov', 'a', 'hard', 'working', 'taxi', 'driver', 'and', 'lyosha', 'a', 'saxophonist', 'develop', 'a', 'bizarre', 'love', 'hate', 'relationship', 'and', 'despite', 'their', 'prejudices', 'realize', 'they', 'aren', 't', 'so', 'different', 'after', 'all'], clean=['shlykov', 'hard', 'working', 'taxi', 'driver', 'lyosha', 'saxophonist', 'develop', 'bizarre', 'love', 'hate', 'relationship', 'despite', 'prejudices', 'realize', 'aren', 'different'], rawFeatures=SparseVector(10000, {135: 1.0, 719: 1.0, 1089: 1.0, 1368: 1.0, 2069: 1.0, 2135: 1.0, 2731: 1.0, 2780: 1.0, 2799: 1.0, 4260: 1.0, 5125: 1.0, 5545: 1.0, 6240: 1.0, 6639: 1.0, 7671: 1.0, 8478: 1.0, 8613: 1.0}), features=SparseVector(10000, {135: 3.6025, 719: 5.

In [0]:

data_set.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               genre|               token|               clean|         rawFeatures|            features|               label|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|23890098|          Taxi Blues|Shlykov, a hard-w...|['World cinema', ...|[shlykov, a, hard...|[shlykov, hard, w...|(10000,[135,719,1...|(10000,[135,719,1...|[1, 0, 0, 0, 0, 1...|
|31186339|    The Hunger Games|"The nation of Pa...| Peeta unexpected...|[the, nation, of,...|[nation, panem, c...|(10000,[47,100,42...|(10000,[47,100,42...|[0, 0, 0, 0, 0, 0...|
|20663735|          Narasimham|Poovalli Induchoo...|['Musical', 'Acti...|[poovalli, induch...|[poovalli, 

In [0]:

labs = list(dic.keys())

cols = data_set.columns
labels = list(dic.keys())

data_set = data_set.rdd.map(lambda row: ([row[c] for c in cols]+row['label'])).toDF(cols+labels)

In [0]:
data_set.show()

+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+------+------------+--------+------+------------+-------------+------+---------------+-----+----------------+---------+-----------+----------+--------------+---------+-------+---------------+-------+---------------+
|movie_id|          movie_name|                plot|               genre|               token|               clean|         rawFeatures|            features|               label|Drama|Comedy|Romance Film|Thriller|Action|World cinema|Crime Fiction|Horror|Black-and-white|Indie|Action/Adventure|Adventure|Family Film|Short Film|Romantic drama|Animation|Musical|Science Fiction|Mystery|Romantic comedy|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+------+---------

In [0]:
trainSet,testSet = data_set.randomSplit([0.7, 0.3], seed = 100)

In [0]:
from pyspark.ml.classification import LogisticRegressionModel
load_models = []
try:
  for i in range(20):
    path = "tf_idf" + "/" +str(i)
    load_models.append(LogisticRegressionModel.load(path))
except: 
  pass

In [0]:
load_models

[LogisticRegressionModel: uid = LogisticRegression_c9a2b790a322, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_87b7bfc05e90, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_ac4514b15bfa, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_323ce43508c5, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_00e595b82d38, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_25d6312daa81, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_0f4bf1399819, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_a96f316c41ca, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_c04a9bdbfc53, numClasses = 2, numFeatures = 10000,
 LogisticRegressionModel: uid = LogisticRegression_c97b9f81b0c3, numClasses = 2, n

In [0]:
from pyspark.ml.classification import LogisticRegression
import time

labels = list(dic.keys())

models = []

if len(load_models)!=20:
  start = time.time()
  for label in labels:
      lr = LogisticRegression(labelCol=label,maxIter=20, regParam=0.3, elasticNetParam=0)
      lrModel = lr.fit(data_set)
      models.append(lrModel)
      end = time.time()
  for index,mdl in enumerate(models):
      mdl.save('/content/tf_idf/'+str(index)+"/")
else:
  models=load_models
end = time.time()

In [0]:
end-start

4826.2507700920105

In [0]:
# for index,mdl in enumerate(models):
#   mdl.save('/content/tf_idf/'+str(index)+"/")

In [0]:
# !zip -r /content/file2.zip /content/tf_idf
# from google.colab import files
# files.download("/content/file2.zip")

In [0]:
##########   Validation Part ##################

# preds = []
# start = time.time()

# for index,model in enumerate(models):
#     print(time.time()-start,index)
#     pred = model.transform(testSet)
#     preds.append(pred.select('prediction').rdd.map(lambda x: x[0]/1.0).collect())
# print(preds)
# end = time.time()
# end-start

In [0]:
##########   Validation Part ##################

# count = len(preds[0])
# final_preds = []
# for i in range(count):
#     final_preds.append([(p[i])/1.0 for p in preds])

In [0]:
##########   Validation Part ##################
# original = testSet.select('label').rdd.map(lambda x: [y/1.0 for y in x[0]]).collect()

In [0]:
##########   Validation Part ##################

# score = []
# for i,v in enumerate(original):
#     score.append((v,final_preds[i]))
# scoreAndLabels = sc.parallelize(score)

In [0]:
##########   Validation Part ##################

# from pyspark.mllib.evaluation import MultilabelMetrics
# metrics = MultilabelMetrics(scoreAndLabels)

In [0]:
##########   Validation Part ##################

# print("Recall = %s" % metrics.recall())
# print("Precision = %s" % metrics.precision())
# print("F1 measure = %s" % metrics.f1Measure())
# print("Accuracy = %s" % metrics.accuracy)

# print("Micro precision = %s" % metrics.microPrecision)
# print("Micro recall = %s" % metrics.microRecall)
# print("Micro F1 measure = %s" % metrics.microF1Measure)

In [0]:
#---------------------------------------------------------------

In [0]:
testing = sqlContext.read.csv('test.csv',header=True)

test_set =model.transform(testing)


In [0]:
test_set.show()


+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|movie_id|          movie_name|                plot|               token|               clean|         rawFeatures|            features|
+--------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
| 1335380|              Exodus|"The film is base...|[the, film, is, b...|[film, based, eve...|(10000,[157,604,8...|(10000,[157,604,8...|
|29062594|A la salida nos v...|A group of teenag...|[a, group, of, te...|[group, teenagers...|(10000,[72,213,37...|(10000,[72,213,37...|
| 9252321|   Come Back, Africa|This story of a Z...|[this, story, of,...|[story, zulu, fam...|(10000,[69,177,23...|(10000,[69,177,23...|
|13455076|       A Merry Mixup|The Stooges play ...|[the, stooges, pl...|[stooges, play, t...|(10000,[324,493,7...|(10000,[324,493,7...|
|24165951|        Getting Even|A soldier-

In [0]:
preds = []
#print(models)
for model in models:
    pred = model.transform(test_set)
    preds.append(pred.select("prediction").rdd.map(lambda x: x[0]).collect())


In [0]:
#preds[0] #.select("prediction").show()

In [0]:
#len(preds[1])
#print(test_set.count())

In [0]:

s = []
for i in range(7777):
  s.append(' '.join([str(int(p[i])) for p in preds]))

sdf = pd.DataFrame(s)
sdf.columns = ['predictions']
# sdf.to_csv('result_tfidf.csv',header=True)

In [0]:
tmp = test_set.select('movie_id').rdd.map(lambda x: x[0]).collect()

In [0]:
mvid = pd.DataFrame(tmp)
mvid.columns = ['movie_id']
# mvid.to_csv('movie_id.csv',header=True)

final_result = mvid.join(sdf, how='outer')
final_result.to_csv('result_tfidf.csv',header=True,index=False)